\

# solucion 2- usando random forest clasifier



In [ ]:
#descomprimir archivos
!ls
!unzip udea-ai4eng-20242.zip

sample_data  udea-ai4eng-20242.zip
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class SaberProPipelineEnhanced(SaberProPipeline):
    def __init__(self):
        super().__init__()
        self.label_encoder = LabelEncoder()

    def create_preprocessing_pipeline(self, df):
        """
        Pipeline de preprocesamiento mejorado
        """
        # Características derivadas
        def create_derived_features(df):
            df = df.copy()
            # Ratio de autofinanciamiento y acceso a internet
            df['RATIO_AUTOFINANCIAMIENTO'] = df['ESTU_PAGOMATRICULAPROPIO'] * df['ESTU_VALORMATRICULAUNIVERSIDAD']
            # Indicador de ventaja socioeconómica
            df['VENTAJA_SOCIOECONOMICA'] = (df['FAMI_ESTRATOVIVIENDA'] +
                                          df['FAMI_TIENEINTERNET'] * 2) / 3
            return df

        # Características numéricas y categóricas
        numeric_features = ['FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
                          'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_TIENEINTERNET',
                          'ESTU_PAGOMATRICULAPROPIO', 'RATIO_AUTOFINANCIAMIENTO',
                          'VENTAJA_SOCIOECONOMICA']

        categorical_features = ['ESTU_PRGM_DEPARTAMENTO']

        # Transformadores con manejo de outliers
        numeric_transformer = Pipeline(steps=[
            ('scaler', RobustScaler())  # Más robusto contra outliers
        ])

        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ])

        # Preprocessor
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # Ensemble de modelos
        estimators = [
            ('rf', RandomForestClassifier(n_estimators=200, random_state=42)),
            ('xgb', XGBClassifier(random_state=42)),
            ('gb', GradientBoostingClassifier(random_state=42))
        ]

        voting_classifier = VotingClassifier(
            estimators=estimators,
            voting='soft'
        )

        # Pipeline completo
        self.pipeline = Pipeline([
            ('feature_engineering', FunctionTransformer(create_derived_features)),
            ('preprocessor', preprocessor),
            ('classifier', voting_classifier)
        ])

        return self.pipeline

    def train_model(self, X_train, y_train):
        """
        Entrenamiento mejorado con validación estratificada
        """
        param_grid = {
            'classifier__rf__n_estimators': [200, 300],
            'classifier__rf__max_depth': [None, 15, 20],
            'classifier__xgb__n_estimators': [200, 300],
            'classifier__xgb__max_depth': [6, 8],
            'classifier__gb__n_estimators': [200, 300],
            'classifier__gb__max_depth': [6, 8]
        }

        # Validación cruzada estratificada
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        grid_search = GridSearchCV(
            self.pipeline,
            param_grid,
            cv=cv,
            scoring=['accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted'],
            refit='f1_weighted',  # Optimizar para F1 ponderado
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)

        print("Mejores parámetros encontrados:")
        print(grid_search.best_params_)

        self.pipeline = grid_search.best_estimator_
        return grid_search.best_score_

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os

class SaberProSubmissionEnhanced:  # Quitamos la herencia por ahora
    def __init__(self):
        self.label_encoder = LabelEncoder()
        self.submission_history = []
        self.model_metrics = {}
        self.pipeline = None

    def fit_transform_target(self, y):
        """
        Codifica las etiquetas del target y guarda el encoder
        """
        return self.label_encoder.fit_transform(y)

    def create_preprocessing_pipeline(self, df):
        """
        Crea el pipeline de preprocesamiento
        """
        # Eliminamos columnas no necesarias
        columns_to_drop = ['ID', 'PERIODO', 'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE', 'ESTU_PRGM_ACADEMICO']

        # Mapeos para variables categóricas
        self.estrato_map = {
            "Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3,
            "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6
        }

        self.horas_trabajo_map = {
            '0': 0, 'Menos de 10 horas': 5,
            'Entre 11 y 20 horas': 15, 'Entre 21 y 30 horas': 25,
            'Más de 30 horas': 30
        }

        self.valor_matricula_map = {
            'Entre 1 millón y menos de 2.5 millones': 1.75,
            'Entre 2.5 millones y menos de 4 millones': 3.25,
            'Menos de 500 mil': 0.25,
            'Entre 500 mil y menos de 1 millón': 0.75,
            'Entre 4 millones y menos de 5.5 millones': 4.75,
            'Más de 7 millones': 7.75,
            'Entre 5.5 millones y menos de 7 millones': 6.25,
            'No pagó matrícula': 0
        }

        # Identificar columnas numéricas y categóricas
        numeric_features = ['FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
                          'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_TIENEINTERNET',
                          'ESTU_PAGOMATRICULAPROPIO']

        categorical_features = ['ESTU_PRGM_DEPARTAMENTO']

        # Crear transformadores
        numeric_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])

        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(drop='first', sparse_output=False))
        ])

        # Combinar transformadores
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # Crear pipeline completo
        self.pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
        ])

        return self.pipeline

    def prepare_data(self, df):
        """
        Prepara los datos aplicando las transformaciones necesarias
        """
        # Copiar el DataFrame
        df_processed = df.copy()

        # Eliminar columnas no necesarias
        columns_to_drop = ['ID', 'PERIODO', 'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE', 'ESTU_PRGM_ACADEMICO']
        df_processed = df_processed.drop(columns_to_drop, axis=1, errors='ignore')

        # Aplicar mapeos
        df_processed['FAMI_ESTRATOVIVIENDA'] = df_processed['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 3').map(self.estrato_map)
        df_processed['ESTU_HORASSEMANATRABAJA'] = df_processed['ESTU_HORASSEMANATRABAJA'].fillna('Más de 30 horas').map(self.horas_trabajo_map)
        df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('Entre 1 millón y menos de 2.5 millones').map(self.valor_matricula_map)
        df_processed['FAMI_TIENEINTERNET'] = df_processed['FAMI_TIENEINTERNET'].fillna('Si').map({'Si': 1, 'No': 0})
        df_processed['ESTU_PAGOMATRICULAPROPIO'] = df_processed['ESTU_PAGOMATRICULAPROPIO'].fillna('No').map({'Si': 1, 'No': 0})

        return df_processed

    def train_model(self, X_train, y_train):
        """
        Entrena el modelo usando GridSearchCV
        """
        param_grid = {
          'classifier__n_estimators': [100],  # Reduced to one value
          'classifier__max_depth': [None, 10], # Reduced to two values
          'classifier__min_samples_split': [2], # Reduced to one value
          'classifier__min_samples_leaf': [1]   # Reduced to one value
}

        grid_search = GridSearchCV(
            self.pipeline,
            param_grid,
            cv=5,
            scoring='accuracy',
            n_jobs=-1,
            verbose=2
        )

        grid_search.fit(X_train, y_train)

        print("Mejores parámetros encontrados:")
        print(grid_search.best_params_)

        self.pipeline = grid_search.best_estimator_
        return grid_search.best_score_

    def prepare_submission(self, test_df, train_df):
        """
        Prepara el archivo de submission con validación y análisis
        """
        # Validación de datos
        self._validate_data(test_df)

        # Preparar datos de test
        test_processed = self.prepare_data(test_df)

        # Obtener predicciones
        predictions = self.pipeline.predict(test_processed)

        # Decodificar predicciones si es necesario
        if hasattr(self, 'label_encoder') and self.label_encoder.classes_.size > 0:
            predictions = self.label_encoder.inverse_transform(predictions)

        # Crear DataFrame de submission
        submission = pd.DataFrame({
            'ID': test_df['ID'],
            'RENDIMIENTO_GLOBAL': predictions
        })

        return submission

    def _validate_data(self, test_df):
        """
        Valida la integridad de los datos de test
        """
        # Verificar columnas requeridas
        required_columns = ['ID', 'FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
                          'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_TIENEINTERNET',
                          'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRGM_DEPARTAMENTO']

        missing_columns = [col for col in required_columns if col not in test_df.columns]
        if missing_columns:
            raise ValueError(f"Columnas faltantes en test_df: {missing_columns}")

    def save_submission(self, submission_df, filename='submission.csv'):
        """
        Guarda el archivo de submission
        """
        # Crear directorio para submissions si no existe
        submission_dir = 'submissions'
        os.makedirs(submission_dir, exist_ok=True)

        # Agregar timestamp al nombre del archivo
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename_with_timestamp = f'submission_{timestamp}.csv'
        filepath = os.path.join(submission_dir, filename_with_timestamp)

        # Guardar submission
        submission_df.to_csv(filepath, index=False)

        print(f"\nSubmission guardada en: {filepath}")
        print("\nPrimeras filas del archivo de submission:")
        print(submission_df.head())

        print("\nDistribución de predicciones:")
        print(submission_df['RENDIMIENTO_GLOBAL'].value_counts())

def main():
    # Cargar datos
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")

    # Separar features y target
    X_train = train_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
    y_train = train_df['RENDIMIENTO_GLOBAL']

    # Crear instancia del pipeline
    pipeline = SaberProSubmissionEnhanced()

    # Codificar variable objetivo
    y_train_encoded = pipeline.fit_transform_target(y_train)

    # Crear y configurar el pipeline
    pipeline.create_preprocessing_pipeline(train_df)

    # Preparar datos de entrenamiento
    X_train_processed = pipeline.prepare_data(X_train)

    # Entrenar modelo
    print("Entrenando modelo...")
    best_score = pipeline.train_model(X_train_processed, y_train_encoded)
    print(f"\nMejor score en validación cruzada: {best_score:.3f}")

    # Generar predicciones
    print("\nGenerando predicciones para el conjunto de test...")
    submission = pipeline.prepare_submission(test_df, train_df)

    # Guardar submission
    pipeline.save_submission(submission)

    return pipeline, submission

if __name__ == "__main__":
    pipeline, submission = main()

Entrenando modelo...
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Mejores parámetros encontrados:
{'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}

Mejor score en validación cruzada: 0.388

Generando predicciones para el conjunto de test...

Submission guardada en: submissions/submission_20241118_041926.csv

Primeras filas del archivo de submission:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo

Distribución de predicciones:
RENDIMIENTO_GLOBAL
bajo          88702
alto          84848
medio-bajo    64203
medio-alto    59033
Name: count, dtype: int64
